In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [27]:
def _load_data(path: str, file_name: str):
    return pd.read_csv(path + file_name)

In [28]:
path = '../../data/raw/'

raw = _load_data(path, 'weather_data.csv')
cols_description = _load_data(path, 'column_descriptions.csv')

In [29]:
pd.set_option('display.max_colwidth', 200)
display(cols_description )

,column,description
0,date,Date and time of the observation.
1,p,Atmospheric pressure in millibars (mbar).
2,T,Air temperature in degrees Celsius (°C).
3,Tpot,"Potential temperature in Kelvin (K), representing the temperature an air parcel would have if moved to a standard pressure level."
4,Tdew,"Dew point temperature in degrees Celsius (°C), indicating the temperature at which air becomes saturated with moisture."
5,rh,"Relative humidity as a percentage (%), showing the amount of moisture in the air relative to the maximum it can hold at that temperature."
6,VPmax,"Maximum vapor pressure in millibars (mbar), representing the maximum pressure exerted by water vapor at the given temperature."
7,VPact,"Actual vapor pressure in millibars (mbar), indicating the current water vapor pressure in the air."
8,VPdef,"Vapor pressure deficit in millibars (mbar), measuring the difference between maximum and actual vapor pressure, used to gauge drying potential."
9,sh,"Specific humidity in grams per kilogram (g/kg), showing the mass of water vapor per kilogram of air."


In [30]:
def _prepare_data(df: pd.DataFrame, date_col: str) -> pd.DataFrame:

    df[date_col] = pd.to_datetime(df[date_col])
    df.set_index(date_col, inplace=True)

    df["hour"] = df.index.hour
    df["month"] = df.index.month
    df["day"] = df.index.day

    return df

raw = _prepare_data(raw, 'date')
raw.head()

,p,T,Tpot,Tdew,rh,VPmax,VPact,VPdef,sh,H2OC,...,wd,rain,raining,SWDR,PAR,max. PAR,Tlog,hour,month,day
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,5.49,...,224.3,0.0,0.0,0.0,0.0,0.0,11.45,0,1,1
2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,5.45,...,206.8,0.0,0.0,0.0,0.0,0.0,11.51,0,1,1
2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,5.43,...,197.1,0.0,0.0,0.0,0.0,0.0,11.60,0,1,1
2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,5.37,...,206.4,0.0,0.0,0.0,0.0,0.0,11.70,0,1,1
2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,5.42,...,209.6,0.0,0.0,0.0,0.0,0.0,11.81,0,1,1


In [31]:
def basic_data_info(data: pd.DataFrame) -> dict:
    """
    Get basic information about the dataset.

    Args:
        data (pd.DataFrame): Input dataframe

    Returns:
        dict: Basic statistics and info
    """
    info = {
        "shape": data.shape,
        "columns": list(data.columns),
        "dtypes": data.dtypes.to_dict(),
        "missing_values": data.isnull().sum().to_dict(),
        "memory_usage": data.memory_usage(deep=True).sum(),
    }
    return info

basic_info = basic_data_info(raw)
print(f"Dataset Shape: {basic_info['shape']}")
print(f"Number of Columns: {len(basic_info['columns'])}")
print(f"Number of Missing Values: {sum(basic_info['missing_values'].values())}")


Dataset Shape: (52696, 23)
Number of Columns: 23
Number of Missing Values: 0


In [40]:
def _basic_statistics(data: pd.DataFrame) -> tuple:
    """
    Compute basic statistics for the numerical columns in the dataframe.
    """
    return data.describe(), data.corr()

describe, correlations = _basic_statistics(raw)

display(describe)
display(correlations)

,p,T,Tpot,Tdew,rh,VPmax,VPact,VPdef,sh,H2OC,...,hour,month,day,rain_total,T_mean,T_min,T_max,rh_mean,SWDR_sum,rain_day
count,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,...,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000,52696.000000
mean,989.989233,10.818241,284.796938,5.409105,72.487133,14.487046,9.676828,4.810131,6.111159,9.782341,...,11.500171,6.513891,15.754497,1.695339,10.818241,6.169912,15.343137,72.487133,18905.526562,0.382572
std,9.207149,7.468671,7.616995,5.956722,19.230260,7.632960,4.023504,5.539320,2.561536,4.082684,...,6.922783,3.451478,8.810597,5.383562,6.703272,6.013112,7.840298,13.967279,13525.979953,0.486020
min,955.580000,-6.440000,266.190000,-13.810000,21.160000,3.770000,2.090000,0.000000,1.300000,2.090000,...,0.000000,1.000000,1.000000,0.000000,-2.705417,-6.440000,-0.120000,43.658750,0.000000,0.000000
25%,984.800000,4.590000,278.550000,0.777500,58.820000,8.480000,6.460000,1.170000,4.070000,6.530000,...,5.000000,4.000000,8.000000,0.000000,4.836875,1.280000,8.820000,61.112292,6754.430000,0.000000
50%,990.920000,10.230000,284.320000,5.260000,75.400000,12.480000,8.890000,2.740000,5.610000,8.990000,...,12.000000,7.000000,16.000000,0.000000,10.582153,5.650000,15.040000,72.993194,14105.500000,0.000000
75%,995.930000,16.180000,290.260000,9.700000,87.900000,18.420000,12.050000,6.440000,7.620000,12.200000,...,18.000000,10.000000,23.000000,0.800000,16.131528,10.840000,21.700000,83.393819,30622.320000,1.000000
max,1020.070000,34.800000,309.130000,20.500000,100.000000,55.670000,24.160000,42.100000,15.400000,24.530000,...,23.000000,12.000000,31.000000,46.400000,26.869375,19.250000,34.800000,100.000000,49617.220000,1.000000


,p,T,Tpot,Tdew,rh,VPmax,VPact,VPdef,sh,H2OC,...,hour,month,day,rain_total,T_mean,T_min,T_max,rh_mean,SWDR_sum,rain_day
p,1.000000,-0.127305,-0.223450,-0.207280,-0.068386,-0.098905,-0.183765,-0.002805,-0.200955,-0.201174,...,-0.014812,-0.154950,0.040475,-0.216411,-0.106707,-0.206373,-0.038445,-0.132758,0.156898,-0.288388
T,-0.127305,1.000000,0.995228,0.782740,-0.540784,0.967901,0.762860,0.779616,0.761550,0.761934,...,0.193087,0.149844,0.003643,0.040782,0.897519,0.836326,0.869004,-0.384982,0.566242,-0.055229
Tpot,-0.223450,0.995228,1.000000,0.789478,-0.524880,0.961013,0.767773,0.766556,0.768193,0.768590,...,0.191264,0.161709,-0.000435,0.061513,0.892399,0.842058,0.857709,-0.365345,0.540917,-0.025764
Tdew,-0.207280,0.782740,0.789478,1.000000,0.087203,0.712665,0.977413,0.272064,0.976878,0.977256,...,0.013741,0.344544,-0.028797,0.184342,0.840526,0.892670,0.759881,0.059886,0.248666,0.125060
rh,-0.068386,-0.540784,-0.524880,0.087203,1.000000,-0.561612,0.085528,-0.836007,0.086812,0.086729,...,-0.295063,0.268277,-0.050200,0.190412,-0.311547,-0.152320,-0.368289,0.726318,-0.571145,0.240820
VPmax,-0.098905,0.967901,0.961013,0.712665,-0.561612,1.000000,0.712554,0.860390,0.711273,0.711482,...,0.192277,0.143217,0.003580,0.011555,0.849436,0.777852,0.833189,-0.381120,0.553485,-0.090241
VPact,-0.183765,0.762860,0.767773,0.977413,0.085528,0.712554,1.000000,0.255506,0.999814,0.999816,...,0.007999,0.308538,-0.023154,0.192119,0.827552,0.880848,0.749457,0.046136,0.254880,0.116861
VPdef,-0.002805,0.779616,0.766556,0.272064,-0.836007,0.860390,0.255506,1.000000,0.253875,0.254162,...,0.259140,-0.026768,0.021749,-0.123630,0.569386,0.432033,0.603723,-0.558685,0.577545,-0.209235
sh,-0.200955,0.761550,0.768193,0.976878,0.086812,0.711273,0.999814,0.253875,1.000000,0.999997,...,0.008172,0.308783,-0.023792,0.196425,0.825580,0.880647,0.746589,0.048809,0.250587,0.121821
H2OC,-0.201174,0.761934,0.768590,0.977256,0.086729,0.711482,0.999816,0.254162,0.999997,1.000000,...,0.008212,0.309186,-0.023779,0.196376,0.825924,0.880965,0.746901,0.048804,0.250708,0.121847


In [41]:
top_corr = correlations.apply(lambda col: col.drop(col.name).abs().sort_values(ascending=False).head(10))
display(top_corr)

,p,T,Tpot,Tdew,rh,VPmax,VPact,VPdef,sh,H2OC,...,hour,month,day,rain_total,T_mean,T_min,T_max,rh_mean,SWDR_sum,rain_day
H2OC,0.201174,NaN,0.768590,0.977256,NaN,NaN,0.999816,NaN,0.999997,NaN,...,NaN,0.309186,0.023779,0.196376,0.825924,0.880965,0.746901,NaN,NaN,NaN
PAR,NaN,NaN,NaN,NaN,0.558661,NaN,NaN,0.576877,NaN,NaN,...,0.031493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SWDR,NaN,NaN,NaN,NaN,0.558295,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.151150
SWDR_sum,NaN,NaN,NaN,NaN,0.571145,NaN,NaN,0.577545,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.786358,NaN,0.346789
T,NaN,NaN,0.995228,0.782740,0.540784,0.967901,0.762860,0.779616,0.761550,0.761934,...,0.193087,NaN,NaN,NaN,0.897519,0.836326,0.869004,0.384982,0.566242,NaN
T_max,NaN,0.869004,0.857709,0.759881,NaN,0.833189,0.749457,0.603723,0.746589,0.746901,...,NaN,NaN,NaN,NaN,0.968229,0.829678,NaN,0.507064,0.715486,0.148248
T_mean,NaN,0.897519,0.892399,0.840526,NaN,0.849436,0.827552,0.569386,0.825580,0.825924,...,NaN,0.166953,NaN,NaN,NaN,0.931820,0.968229,0.428941,0.630897,NaN
T_min,0.206373,0.836326,0.842058,0.892670,NaN,0.777852,0.880848,NaN,0.880647,0.880965,...,NaN,0.233625,NaN,NaN,0.931820,NaN,0.829678,NaN,NaN,NaN
Tdew,0.207280,0.782740,0.789478,NaN,NaN,0.712665,0.977413,NaN,0.976878,0.977256,...,NaN,0.344544,0.028797,0.184342,0.840526,0.892670,0.759881,NaN,NaN,NaN
Tlog,NaN,0.981562,0.972736,0.741963,0.558603,0.962748,0.727446,0.798241,0.725602,0.725934,...,0.203697,NaN,NaN,NaN,0.868541,0.787411,0.856469,0.388493,0.590245,NaN


In [ ]:
def _daily_columns(df: pd.DataFrame) -> pd.DataFrame:
    rain_day = df.groupby(df.index.date).agg(
        rain_sum=('rain', 'sum'),
        T_mean=('T', 'mean'),
        T_min=('T', 'min'),
        T_max=('T', 'max'),
        rh_mean=('rh', 'mean'),
    SWDR_sum=('SWDR', 'sum')
)

    # Add daily rain sum to original dataframe
    date_index = pd.Series(df.index.date, index=df.index)
    df['rain_total'] = date_index.map(rain_day['rain_sum'])
    df['T_mean'] = date_index.map(rain_day['T_mean'])
    df['T_min'] = date_index.map(rain_day['T_min'])
    df['T_max'] = date_index.map(rain_day['T_max'])
    df['rh_mean'] = date_index.map(rain_day['rh_mean'])
    df['SWDR_sum'] = date_index.map(rain_day['SWDR_sum'])

    df['rain_day'] = df['rain_total'].apply(lambda x: 1 if x > 0 else 0)

    return df

raw = _daily_columns(raw)

,p,T,Tpot,Tdew,rh,VPmax,VPact,VPdef,sh,H2OC,...,hour,month,day,rain_total,T_mean,T_min,T_max,rh_mean,SWDR_sum,rain_day
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,5.49,...,0,1,1,0.0,-0.514196,-3.46,4.58,86.429301,8250.91,0
2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,5.45,...,0,1,1,0.0,-0.514196,-3.46,4.58,86.429301,8250.91,0
2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,5.43,...,0,1,1,0.0,-0.514196,-3.46,4.58,86.429301,8250.91,0
2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,5.37,...,0,1,1,0.0,-0.514196,-3.46,4.58,86.429301,8250.91,0
2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,5.42,...,0,1,1,0.0,-0.514196,-3.46,4.58,86.429301,8250.91,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 23:20:00,978.32,2.28,277.16,-0.80,80.0,7.20,5.76,1.44,3.67,5.89,...,23,12,31,0.0,1.870139,-0.07,3.43,83.545694,3902.74,0
2020-12-31 23:30:00,978.30,2.13,277.01,-0.43,83.1,7.12,5.92,1.20,3.77,6.05,...,23,12,31,0.0,1.870139,-0.07,3.43,83.545694,3902.74,0
2020-12-31 23:40:00,978.26,1.99,276.88,-0.71,82.2,7.05,5.80,1.26,3.69,5.93,...,23,12,31,0.0,1.870139,-0.07,3.43,83.545694,3902.74,0


In [ ]:
def _coordinate_hourly_monthly_data(df: pd.DataFrame) -> pd.DataFrame:
    df['hour_sin'] = np.sin(2 * np.pi * df.index.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df.index.hour / 24)
    df['month_sin'] = np.sin(2 * np.pi * df.index.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df.index.month / 12)
    return df

raw = _coordinate_hourly_monthly_data(raw)


In [45]:
raw.resample('D').mean().describe()[['rain_total', 'T_mean', 'T_min', 'T_max', 'rh_mean', 'wv']]

,rain_total,T_mean,T_min,T_max,rh_mean,wv
count,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000
mean,1.690463,10.794832,6.159019,15.307439,72.510647,1.992138
std,5.383870,6.718604,6.016605,7.870443,13.978555,3.701414
min,0.000000,-2.705417,-6.440000,-0.120000,43.658750,-65.882431
25%,0.000000,4.836076,1.340000,8.815000,61.169688,1.452118
50%,0.000000,10.582153,5.650000,15.040000,72.993194,1.879653
75%,0.800000,16.074028,10.735000,21.690000,83.360451,2.685799
max,46.400000,26.869375,19.250000,34.800000,100.000000,6.453403


In [ ]:
column_descriptions = {
    "date": "Date and time of the observation.",
    "p": "Atmospheric pressure in millibars (mbar).",
    "T": "Air temperature in degrees Celsius (°C).",
    "Tpot": "Potential temperature in Kelvin (K), representing the temperature an air parcel would have if moved to a standard pressure level.",
    "Tdew": "Dew point temperature in degrees Celsius (°C), indicating the temperature at which air becomes saturated with moisture.",
    "rh": "Relative humidity as a percentage (%), showing the amount of moisture in the air relative to the maximum it can hold at that temperature.",
    "VPmax": "Maximum vapor pressure in millibars (mbar), representing the maximum pressure exerted by water vapor at the given temperature.",
    "VPact": "Actual vapor pressure in millibars (mbar), indicating the current water vapor pressure in the air.",
    "VPdef": "Vapor pressure deficit in millibars (mbar), measuring the difference between maximum and actual vapor pressure, used to gauge drying potential.",
    "sh": "Specific humidity in grams per kilogram (g/kg), showing the mass of water vapor per kilogram of air.",
    "H2OC": "Concentration of water vapor in millimoles per mole (mmol/mol) of dry air.",
    "rho": "Air density in grams per cubic meter (g/m³), reflecting the mass of air per unit volume.",
    "wv": "Wind speed in meters per second (m/s), measuring the horizontal motion of air.",
    "max. wv": "Maximum wind speed in meters per second (m/s), indicating the highest recorded wind speed over the period.",
    "wd": "Wind direction in degrees (°), representing the direction from which the wind is blowing.",
    "rain": "Total rainfall in millimeters (mm), showing the amount of precipitation over the observation period.",
    "raining": "Duration of rainfall in seconds (s), recording the time for which rain occurred during the observation period.",
    "SWDR": "Short-wave downward radiation in watts per square meter (W/m²), measuring incoming solar radiation.",
    "PAR": "Photosynthetically active radiation in micromoles per square meter per second (µmol/m²/s), indicating the amount of light available for photosynthesis.",
    "max. PAR": "Maximum photosynthetically active radiation recorded in the observation period in µmol/m²/s.",
    "Tlog": "Temperature logged in degrees Celsius (°C), potentially from a secondary sensor or logger.",
    "OT": "Likely refers to an operational timestamp or an offset in time, but may need clarification depending on the dataset's context."
}
